# Chapter3 基礎技術

## 3.1 
> 3.1節にあるように, 実際に形態素解析を行ってみよう. 3.1節と同じ結果が出ただろうか. MeCabを使った形態素解析においては, どの辞書を使うかによって解析結果が異なってくる. MeCabの辞書としてどのようなものがあるか調べてみよう. また辞書を変えたとき, 解析結果がどのように変わるか調べてみよう.

In [ ]:
!apt update 
!apt install mecab libmecab-dev mecab-ipadic-utf8 mecab-jumandic-utf8

In [1]:
import subprocess

In [ ]:
subprocess.run("mecab", input="日本語の文の形態素解析を行った.", encoding="utf-8")

In [2]:
subprocess.run(["mecab", "-D"])

filename:	/var/lib/mecab/dic/debian/sys.dic
version:	102
charset:	UTF-8
type:	0
size:	392127
left size:	1316
right size:	1316



CompletedProcess(args=['mecab', '-D'], returncode=1)

In [11]:
subprocess.run(["ls", "/var/lib/mecab/dic"])

debian
ipadic
ipadic-utf8
juman-utf8


CompletedProcess(args=['ls', '/var/lib/mecab/dic'], returncode=0)

In [3]:
subprocess.run(["mecab", "-d", "/var/lib/mecab/dic/ipadic-utf8"], input="日本語の文の形態素解析を行った.", encoding="utf-8")

日本語	名詞,一般,*,*,*,*,日本語,ニホンゴ,ニホンゴ
の	助詞,連体化,*,*,*,*,の,ノ,ノ
文	名詞,一般,*,*,*,*,文,ブン,ブン
の	助詞,連体化,*,*,*,*,の,ノ,ノ
形態素	名詞,一般,*,*,*,*,形態素,ケイタイソ,ケイタイソ
解析	名詞,サ変接続,*,*,*,*,解析,カイセキ,カイセキ
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
行っ	動詞,自立,*,*,五段・ワ行促音便,連用タ接続,行う,オコナッ,オコナッ
た	助動詞,*,*,*,特殊・タ,基本形,た,タ,タ
.	名詞,サ変接続,*,*,*,*,*
EOS


CompletedProcess(args=['mecab', '-d', '/var/lib/mecab/dic/ipadic-utf8'], returncode=0)

In [6]:
subprocess.run(["mecab", "-d", "/var/lib/mecab/dic/juman-utf8"], input="日本語の文の形態素解析を行った.", encoding="utf-8")

日本	名詞,地名,*,*,日本,にほん,代表表記:日本/にほん 地名:国
語	名詞,普通名詞,*,*,語,ご,代表表記:語/ご 漢字読み:音 カテゴリ:抽象物
の	助詞,接続助詞,*,*,の,の,*
文	名詞,普通名詞,*,*,文,ぶん,代表表記:文/ぶん 漢字読み:音 カテゴリ:抽象物
の	助詞,接続助詞,*,*,の,の,*
形態	名詞,普通名詞,*,*,形態,けいたい,代表表記:形態/けいたい カテゴリ:形・模様
素	名詞,普通名詞,*,*,素,そ,代表表記:素/そ 漢字読み:音 カテゴリ:抽象物
解析	名詞,サ変名詞,*,*,解析,かいせき,代表表記:解析/かいせき カテゴリ:抽象物 ドメイン:教育・学習;科学・技術
を	助詞,格助詞,*,*,を,を,*
行った	動詞,*,子音動詞ワ行,タ形,行う,おこなった,代表表記:行う/おこなう
.	特殊,記号,*,*,*,*,*
EOS


CompletedProcess(args=['mecab', '-d', '/var/lib/mecab/dic/juman-utf8'], returncode=0)

## 3.2
> 新聞記事など, 身近なテキストをダウンロードし, それを形態素解析してみよう. その上で, 実現する形態素ごとにその形態素が出現する頻度を数えるプログラムを書いてみよう. その結果は元のテキストの特徴を表しているだろうか.

In [19]:
import spacy
import requests
import re
from collections import defaultdict

In [74]:
neko = requests.get('https://nlp100.github.io/data/neko.txt').text
nlp = spacy.load('ja_ginza')
nekos = re.split(r"\r\n\r\n", neko)

In [75]:
print(len(nekos))
for i in nekos:
    if len(i) < 10:
        print(len(i), i[:4])

710
1 一
3 
二
3 
三
3 
四
3 
五
3 
六
3 
七
3 
八
3 
九
3 
十
4 
十一
0 


In [76]:
docs = []
for i in nekos:
    if len(i) < 10:
        continue
    docs.append(nlp(i))

In [78]:
word_count = defaultdict(int)

for doc in docs:
    for token in doc:
        word_count[token.text] += 1
sorted_count = sorted(word_count.items(), key = lambda d: d[1], reverse=True)
sorted_count[:10]

[('の', 9545),
 ('\r\n', 8399),
 ('。', 7486),
 ('て', 7401),
 ('に', 7047),
 ('、', 6773),
 ('は', 6484),
 ('と', 6149),
 ('を', 6118),
 ('が', 5395)]

In [80]:
pos_word_count = dict()

for doc in docs:
    for token in doc:
        if token.pos_ in pos_word_count.keys():
            pos_word_count[token.pos_][token.text] += 1
        else:
            pos_word_count[token.pos_] = defaultdict(int)

for k, v in pos_word_count.items():
    print(k)
    sorted_count_temp = sorted(v.items(), key = lambda d: d[1], reverse=True)
    print(sorted_count_temp[:10])

SYM
[('—', 666), ('…', 433), ('\u3000', 261), ('\r\n\u3000', 135), ('・', 36), ('※', 34), ('○', 22), ('っ', 20), ('アンドレア・デル・サルト', 4), ('ッ', 4)]
NOUN
[('\r\n', 7539), ('事', 1213), ('もの', 960), ('主人', 933), ('御', 551), ('君', 541), ('吾輩', 471), ('人', 457), ('時', 349), ('ところ', 342)]
ADP
[('の', 7555), ('は', 6478), ('を', 6118), ('に', 5819), ('と', 5038), ('が', 4356), ('も', 3218), ('で', 1933), ('へ', 995), ('から', 959)]
AUX
[('た', 3969), ('だ', 2701), ('で', 2215), ('ない', 1652), ('し', 1413), ('な', 1286), ('に', 1141), ('です', 978), ('ん', 845), ('する', 618)]
VERB
[('ある', 1713), ('いる', 1255), ('し', 1033), ('云う', 935), ('なる', 455), ('する', 435), ('なっ', 406), ('い', 396), ('来', 374), ('見', 364)]
PUNCT
[('。', 7485), ('、', 6773), ('「', 3238), ('」', 3238), ('？', 141), ('\r\n', 9), ('！', 8), ('\u3000', 4), ('(', 2), (')', 2)]
ADV
[('どう', 411), ('そう', 271), ('少し', 197), ('また', 188), ('ちょっと', 171), ('もう', 151), ('こう', 117), ('まだ', 117), ('よく', 90), ('まあ', 89)]
ADJ
[('ない', 771), ('\r\n', 762), ('いい', 249), ('なく', 1

## 3.3
> 口語的な文では係り受けに交差が生じることがある. 係り受けに交差が生じている文を考えてみよう. またそのような文を解析するためにはどのような考え方が必要になるか, 考えてみよう.

In [70]:
from spacy import displacy

In [71]:
example = "うなぎを浜松に食べに行く"
doc = nlp(example)
displacy.render(doc, style='dep')

In [73]:
example = "うなぎを浜松に食べるために行く"
doc = nlp(example)
displacy.render(doc, style='dep')

## 3.4
> 新聞記事などから一つ文を選び, その文の修辞構造木を作成してみよう. 作成したら, 友人などもの同じように修辞構造木を作成してもらい, 自分のものと比較してみよう. 修辞構造木は注釈付けが難しいことが知られており, 同じ文に対しても注釈者によって別の構造が付与されることがままある. その原因についても考えてみよう.

Skip